# Adding records

The `addRecord()` method creates a JSON string with information about your Senzing version.

## Prepare environment

### Initialize python environment

In [ ]:
import os
import sys
import json

# For RenderJSON

import uuid
from IPython.display import display_javascript, display_html, display

### Helper class for JSON rendering

A class for pretty-printing JSON.
Not required by Senzing, 
but helps visualize JSON.

In [ ]:
class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        elif isinstance(json_data, bytearray):
            self.json_str = json_data.decode()
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height:100%; width:100%; background-color: LightCyan"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
        document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

### System path

Update system path.

In [ ]:
python_path = "{0}/python".format(
    os.environ.get("SENZING_G2_DIR", "/opt/senzing/g2"))
sys.path.append(python_path)

### Initialize variables

Create variables used for G2Engine.

In [ ]:
%run senzing-init-config.ipynb

In [ ]:
%store -r senzing_config_json
%store -r config_id_bytearray
config_id=config_id_bytearray.decode()

In [ ]:
RenderJSON(senzing_config_json)

## G2Engine

The G2Engine API...

In [ ]:
from senzing import G2Engine, G2Exception

## Initialization

To start using Senzing G2Engine, create and initialize an instance.
This should be done once per process.
The `init()` method accepts the following parameters:

- **module_name:** A short name given to this instance of the G2 engine (i.e. your G2Module object)
- **g2module_ini_pathname:** A fully qualified path to the G2 engine INI file (often /opt/senzing/g2/python/G2Module.ini)
- **verbose_logging:** A boolean which enables diagnostic logging - this will print a massive amount of information to stdout (default = False)
- **config_id:** (optional) The identifier value for the engine configuration can be returned here.

Calling this function will return "0" upon success.

In [ ]:
g2_engine = G2Engine()
g2_engine.init(module_name, senzing_config_json, verbose_logging, config_id)

## Prime Engine

The `primeEngine()` method may optionally be called to pre-initialize some of the heavier weight internal resources of the G2 engine.

In [ ]:
response = g2_engine.primeEngine()

## addRecord()

Once the Senzing engine is initialized, use addRecord() to load a record into the Senzing repository -- addRecord() can be called as many times as desired and from multiple threads at the same time. The addRecord() function returns "0" upon success, and accepts four parameters as input:

- **datasource_code:** The name of the data source the record is associated with. This value is configurable to the system
- **record_id:** The record ID, used to identify distinct records
- **data_string:** A JSON document with the attribute data for the record
- **load_id:** The observation load ID for the record; value can be null and will default to data_source


In [ ]:
datasource_code = "TEST"
record_id = "1"
load_id = None
data = {
	"NAMES": [{
		"NAME_TYPE": "PRIMARY",
		"NAME_LAST": "Smith",
		"NAME_FIRST": "John",
		"NAME_MIDDLE": "M"
	}],
	"PASSPORT_NUMBER": "PP11111",
	"PASSPORT_COUNTRY": "US",
	"DRIVERS_LICENSE_NUMBER": "DL11111",
	"SSN_NUMBER": "111-11-1111"
}
data_string = json.dumps(data)

try:
    g2_engine.addRecord(datasource_code, record_id, data_string, load_id)

except G2Exception as err:
    print(g2_engine.getLastException())

## getRecord()

Use `getRecord()` to retrieve a single record from the data repository; the record is assigned in JSON form to a user-designated buffer, and the function itself returns "0" upon success. Once the Senzing engine is initialized, `getRecord()` can be called as many times as desired and from multiple threads at the same time. The `getRecord()` function accepts the following parameters as input:

- **data_source:** The name of the data source the record is associated with. This value is configurable to the system
- **record_id:** The record ID, used to identify the record for retrieval
- **response:** A memory buffer for returning the response document; if an error occurred, an error response is stored here

In [ ]:
response_bytearray = bytearray()

try:
    g2_engine.getRecord(
        datasource_code,
        record_id,
        response_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

## getEntityByRecordID()

Use `getEntityByRecordID()` to retrieve entity data based on the record ID of a particular data record. This function accepts the following parameters as input:

- **datasource_code:** The name of the data source the record is associated with. This value is configurable to the system
- **record_id:** The record ID for a particular data record
- **response:** A memory buffer for returning the response document; if an error occurred, an error response is stored here.

In [ ]:
response_list = []
result = g2_engine.getEntityByRecordID(datasource_code, record_id, response_list)

response_string = "".join(response_list)
response_dictionary = json.loads(response_string)
response = json.dumps(response_dictionary, sort_keys=True, indent=4)
print("Result: {0}\n{1}".format(result, response))

## getEntityByEntityID()

Use `getEntityByEntityID()` to retrieve entity data based on the ID of a resolved identity. This function accepts the following parameters as input:

- **entity_id:** The numeric ID of a resolved entity
- **response:** A memory buffer for returning the response document; if an error occurred, an error response is stored here.

In [ ]:
entity_id = 1
response_bytearray = bytearray()

try:
    g2_engine.getEntityByEntityID(entity_id, response_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)

## searchByAttributes()

Use `searchByAttributes()` to retrieve entity data based on a user-specified set of entity attributes. This function accepts the following parameters as input:

- **data_string:** A JSON document with the attribute data to search for
- **response:** A memory buffer for returning the response document; if an error occurred, an error response is stored here.

In [ ]:
response_bytearray = bytearray()

try:
    g2_engine.searchByAttributes(data_string, response_bytearray)

except G2Exception as err:
    print(g2_engine.getLastException())
RenderJSON(response_bytearray)